In [1]:
file = "calories.csv"

Structure:
    
- get some food data
- make food object, bare classes showing dunder (magic) methods (init, str/repr, compare, add)
- food log class to show more dunders: (len, iter, create a context manager)
- create some functions using the food objects
- add exception handling (raise, catch, own exception, best practices)

In [2]:
import csv

In [3]:
with open(file) as f:
    for row in csv.DictReader(f):
        print(row)

{'Food': '1000 Island,Salad Drsng,Local', 'Measure': '1 Tbsp', 'Weight (g)': '15', 'kCal': '25', 'Fat (g)': '2', 'Carbo(g)': '2', 'Protein (g)': '0'}
{'Food': '1000 Island,Salad Drsng,Reglr', 'Measure': '1 Tbsp', 'Weight (g)': '16', 'kCal': '60', 'Fat (g)': '6', 'Carbo(g)': '2', 'Protein (g)': '0'}
{'Food': "40% Bran Flakes,Kellogg's", 'Measure': '1 oz', 'Weight (g)': '28.35', 'kCal': '90', 'Fat (g)': '1', 'Carbo(g)': '22', 'Protein (g)': '4'}
{'Food': '40% Bran Flakes,Post', 'Measure': '1 oz', 'Weight (g)': '28.35', 'kCal': '90', 'Fat (g)': '0', 'Carbo(g)': '22', 'Protein (g)': '3'}
{'Food': 'Alfalfa Seeds,Sprouted,Raw', 'Measure': '1 Cup', 'Weight (g)': '33', 'kCal': '10', 'Fat (g)': '0', 'Carbo(g)': '1', 'Protein (g)': '1'}
{'Food': 'All-Bran Cereal', 'Measure': '1 oz', 'Weight (g)': '28.35', 'kCal': '70', 'Fat (g)': '1', 'Carbo(g)': '21', 'Protein (g)': '4'}
{'Food': 'Almonds,Slivered', 'Measure': '1 Cup', 'Weight (g)': '135', 'kCal': '795', 'Fat (g)': '70', 'Carbo(g)': '28', 'Prot

In [87]:
from functools import total_ordering


@total_ordering
class Food:
    def __init__(self, name, measure, weight, kcal):
        self.name = name
        self.measure = measure
        self.weight = weight
        self.kcal = kcal
    
    def __repr__(self):
        cls_name = type(self).__name__
        return f"{cls_name}('{self.name}', '{self.measure}', {self.weight}, {self.kcal})"
    
    def __str__(self):
        return self.name
    
    def __eq__(self, other):
        return self.kcal == other.kcal
    
    def __lt__(self, other):
        return self.kcal < other.kcal
    
    def __add__(self, other):          
        name = f"{self.name} & {other.name}"
        
        if self.measure != other.measure:
            raise ValueError("Measures should be the same")
            
        weight = self.weight + other.weight
        kcal = self.kcal + other.kcal
        
        cls = type(self)
        return cls(name, self.measure, weight, kcal)
    

In [88]:
pie = Food('Apple Pie', '100 grams', 158, 405)

In [89]:
bananas = Food("Bananas", "1 Piece", 114, 105)

In [90]:
bananas > pie

False

In [91]:
banana_apple_pie = bananas + pie

ValueError: Measures should be the same

In [92]:
car = Car()

In [93]:
bananas + car

AttributeError: 'Car' object has no attribute 'name'

In [125]:
from dataclasses import dataclass


@dataclass
class Food:
    name: str
    measure: str
    weight: int
    kcal: int
        
    def __str__(self):
        return self.name

    def __eq__(self, other):
        return self.kcal == other.kcal
    
    def __lt__(self, other):
        return self.kcal < other.kcal
    
    def __add__(self, other):          
        name = f"{self.name} & {other.name}"
        
        if self.measure != other.measure:
            raise ValueError("Measures should be the same")
            
        weight = self.weight + other.weight
        kcal = self.kcal + other.kcal
        
        cls = type(self)
        return cls(name, self.measure, weight, kcal)

In [126]:
pie = Food('Apple Pie', '100 grams', 158, 405)

In [127]:
bananas = Food("Bananas", "1 Piece", 114, 105)

In [129]:
pie > bananas

True

In [215]:
from dataclasses import field

@dataclass
class FoodLog:
    owner: str
    kcal_limit: int
    _foods: list[Food] = field(default_factory=list)
    index: int = 0
        
    def add_food(self, food: Food):
        self._foods.append(food)
    
    @property
    def total_calories_consumed(self):
        return sum(food.kcal for food in self._foods)
    
    @property
    def calories_to_consume_today(self):
        return self.kcal_limit - self.total_calories_consumed
    
    def __len__(self):
        return len(self._foods)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index < len(self._foods):
            self.index += 1
            return self._foods[self.index - 1]
        else:
            raise StopIteration

In [216]:
tracker = FoodLog("bob", 2500)

In [217]:
tracker.add_food(pie)
tracker.add_food(pie)
tracker.add_food(bananas)

In [218]:
tracker._foods

[Food(name='Apple Pie', measure='100 grams', weight=158, kcal=405),
 Food(name='Apple Pie', measure='100 grams', weight=158, kcal=405),
 Food(name='Bananas', measure='1 Piece', weight=114, kcal=105)]

In [219]:
len(tracker)

3

In [220]:
for food in tracker: print(food)

Apple Pie
Apple Pie
Bananas


In [211]:
next(tracker)

Food(name='Apple Pie', measure='100 grams', weight=158, kcal=405)

In [212]:
next(tracker)

Food(name='Apple Pie', measure='100 grams', weight=158, kcal=405)

In [213]:
next(tracker)

Food(name='Bananas', measure='1 Piece', weight=114, kcal=105)

In [214]:
try:
    next(tracker)
except StopIteration:
    # handle

StopIteration: 